# Synthea Case Study
*Lab 26 April 2022*

* https://synthea.mitre.org/
* https://github.com/synthetichealth/synthea

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Patients;

CREATE TABLE Patients(
    id VARCHAR(36),
    birthdate DATE,
    deathdate DATE,
    ssn VARCHAR(11),
    drivers VARCHAR(9),
    passport VARCHAR(15),
    prefix VARCHAR(4),
    first VARCHAR(20),
    last VARCHAR(20),
    suffix VARCHAR(5),
    maiden VARCHAR(20),
    marital CHAR,
    race VARCHAR(10),
    ethnicity VARCHAR(15),
    gender CHAR,
    birthplace VARCHAR(100),
    address VARCHAR(50),
    city VARCHAR(30),
    state VARCHAR(30),
    county VARCHAR(50),
    zip VARCHAR(10),
    lat DECIMAL(18,15),
    lon DECIMAL(18,15),
    healthcare_expenses VARCHAR(20),
    healthcare_coverage VARCHAR(20),
    PRIMARY KEY(id)
) AS SELECT id,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,
            birthplace,address,city,state,county,zip,lat,lon,healthcare_expenses,healthcare_coverage
FROM CSVREAD('../../data/synthea/scenario01/csv/patients.csv');

DROP TABLE IF EXISTS Encounters;

CREATE TABLE Encounters(
    id VARCHAR(36),
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    organization VARCHAR(36),
    provider VARCHAR(36),
    payer  VARCHAR(36),
    encounterclass VARCHAR(20),
    code VARCHAR(15),
    description VARCHAR(100),
    base_encounter_cost DECIMAL(8,2),
    total_claim_cost DECIMAL(8,2),
    payer_coverage DECIMAL(8,2),
    reasoncode VARCHAR(15),
    reasondescription VARCHAR(100),
    PRIMARY KEY(id)
) AS SELECT id,start,stop,patient,organization,provider,payer,encounterclass,code,description,base_encounter_cost,
            total_claim_cost,payer_coverage,reasoncode,reasondescription
FROM CSVREAD('../../data/synthea/scenario01/csv/encounters.csv');

DROP TABLE IF EXISTS Conditions;

CREATE TABLE Conditions(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    description_condition VARCHAR(100),
) AS SELECT start,stop,patient,encounter,code,description
FROM CSVREAD('../../data/synthea/scenario01/csv/conditions.csv');

In [18]:
select * from Encounters
limit 100

In [3]:
DROP TABLE IF EXISTS PatientCondition;

CREATE TABLE PatientCondition (
    patient VARCHAR(100),
    birthdate DATETIME,
    deathdate DATETIME,
    condition_description VARCHAR(100)
) AS
SELECT p.Id, p.birthdate, p.deathdate, c.description_condition
    FROM Patients p
    JOIN Conditions c on p.Id = c.patient;

In [4]:
SELECT * FROM PatientCondition
LIMIT 100;

In [13]:
CALL CSVWRITE('../../data/synthea/scenario01/csv-ml/patient-condition.csv', 'select * from PatientCondition')

36396

In [32]:
CREATE TABLE Procedures(
    start DATETIME,
    stop DATETIME,
    patient VARCHAR(36),
    encounter VARCHAR(36),
    code VARCHAR(20),
    procedure_description VARCHAR(120),
    reasoncode VARCHAR(36),
    reason_description VARCHAR(120)
) AS SELECT start,stop,patient,encounter,code,description, reasoncode, reasondescription
FROM CSVREAD('../../data/synthea/scenario01/csv/procedures.csv');

In [12]:
CALL CSVWRITE('../../data/synthea/scenario01/csv-ml/procedure.csv', 'select * from Procedure')

108206

In [6]:
SELECT * FROM Procedure
LIMIT 100;

In [7]:
DROP TABLE IF EXISTS PatientProcedure;

CREATE TABLE PatientProcedure (
    patient VARCHAR(100),
    birthdate DATETIME,
    deathdate DATETIME,
    procedure_description VARCHAR(120),
    reason_description VARCHAR(120)
) AS
SELECT p.Id, p.birthdate, p.deathdate, c.procedure_description, c.reason_description
    FROM Patients p
    JOIN Procedure c on p.Id = c.patient;

In [16]:
select * from PatientProcedure
limit 100

In [11]:
CALL CSVWRITE('../../data/synthea/scenario01/csv-ml/patient-procedure.csv', 'select * from PatientProcedure')

108206

In [8]:
SELECT COUNT (DISTINCT patient) FROM PatientProcedure WHERE procedure_description LIKE '%Depression%';

981

In [9]:
DROP TABLE IF EXISTS PatientEncounter;

CREATE TABLE PatientEncounter (
    patient VARCHAR(100),
    birthdate DATETIME,
    deathdate DATETIME,
    code VARCHAR(120),
    start VARCHAR(120),
    stop VARCHAR(120),
    description VARCHAR(120),
    reason_description VARCHAR(120)
) AS
SELECT p.Id, p.birthdate, p.deathdate, e.code, e.start, e.stop, e.description, e.reasondescription
    FROM Patients p
    JOIN Encounters e on p.Id = e.patient;

In [13]:
select * from PatientEncounter
limit 100

In [10]:
CALL CSVWRITE('../../data/synthea/scenario01/csv-ml/patient-encounter.csv', 'select * from PatientEncounter')

59481

In [39]:
DROP TABLE IF EXISTS DepressionDeath;

CREATE TABLE DepressionDeath (
    patient VARCHAR(36),
    last_encounter DATETIME,
    deathdate DATETIME,
    procedure_description VARCHAR(120),
    depression_screening SMALLINT DEFAULT 0
) AS
SELECT p.id, MAX(e.start) last_encounter, p.deathdate, r.procedure_description, 0
    FROM Patients p
    JOIN Encounters e ON p.id = e.patient
    JOIN Procedures r ON e.id = r.encounter
    GROUP BY p.id, r.procedure_description;

In [44]:
UPDATE DepressionDeath
SET depression_screening = 1
WHERE procedure_description LIKE '%Depression%';

In [45]:
select * from DepressionDeath
limit 100

In [47]:
SELECT COUNT (DISTINCT patient) FROM DepressionDeath WHERE depression_screening = 1

981

In [48]:
CALL CSVWRITE('../../data/synthea/scenario01/csv-ml/depression-death.csv', 'select * from DepressionDeath')

21190